<a href="https://colab.research.google.com/github/poporubeus/Chess_network/blob/main/Alex_Sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from requests import Session
from bs4 import BeautifulSoup
from urllib.parse import urlparse

Funzione che prende link alla pagina principale dei tornei e restituisce in output lista con tutte le pagine di tutti gli anni

In [1]:


session = Session()


# tournaments home https://www.365chess.com/tournaments.php


def get_tournaments_pages(url, year):
    lista_link=[]
    
    response = session.get(
        url=url,
        headers={
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,it;q=0.7',
            'Connection': 'keep-alive',
            'Host': 'www.365chess.com',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        }
    )

    if response:
        #print("Get in Tournaments Page")
        base_url = f'https://{urlparse(response.url).hostname}/'
        soup = BeautifulSoup(response.text, 'html.parser')
        pages = soup.find('table', {'class': 'pag'})
        if pages and pages.td and pages.td.findAll('a'):
            # collect all pages

            # adding first page
            pages_dict = {
                # number: url
                '1': response.url  # or tournament_url
            }
            for a in [a for a in pages.td.findAll('a') if a.attrs and a.attrs.get('href') and not a.text == 'Next']:
                # number: url
                pages_dict[a.text] = base_url + a.attrs.get('href')

            last_page_number = int(list(pages_dict.keys())[-1])
            if last_page_number != len(pages_dict):
                # fix hidden pages
                pages_dict = {'1': f'{base_url}?p=1&tname=&year={year}&country=&avgelo='}
                count_start = 0
                for num in range(2, last_page_number + 1):
                    count_start += 60
                    pages_dict[str(num)] = f'{base_url}tournaments.php?p=1&tname=&year={year}&country=&avgelo=&start={count_start}'
            
         

            # print all pages
            for page_number, page_url in pages_dict.items():
                lista_link.append(page_url)
               
    
    return(lista_link)


Qui viene fatto un loop sugli anni e si costruisce una lista di link. Vorrei in qualche maniera però averli raggruppati per anni. Non so fino a quanto possa essere un problema ma bon, vi avviso

In [4]:
start = 2010
finish = 2012
years = list(range(start,finish))
lista_pagine=[]
#questo è il loop sugli anni che salva in una lista
for selected_year in years:
  lista_pagine.append(get_tournaments_pages(f'https://www.365chess.com/tournaments.php?year={selected_year}', selected_year)) 
#questo linearifica la lista di liste in una lista di oggetti
lista_pagine_linear = [item for sublist in lista_pagine for item in sublist]


Ora, dagli url per gli anni, cerco di ottenere gli url dei tornei

In [21]:
def get_tournaments(url):
  lista2=[]
  soup = BeautifulSoup(requests.get(str(url)).text, 'html.parser')
  pages = soup.find('table', {'class': 'hoverable stable'})
  for a in pages.find_all('a', href=True):
    lista2.append(a['href'])
  return(lista2)

  

qui provo che funge e funge. Manca capire come muoversi tra le pagine

In [22]:
linkissimo=[]
for link in lista_pagine_linear:
    linkissimo.append(get_tournaments(link))
print(linkissimo)

[['https://www.365chess.com/tournaments/53rd_Masters_2010/38881', 'https://www.365chess.com/tournaments/WSB_Masters_IM_2010/38926', 'https://www.365chess.com/tournaments/WSB_Masters_GM_2010/38924', 'https://www.365chess.com/tournaments/Rilton_Cup_2010-11_2010/38882', 'https://www.365chess.com/tournaments/I_Gipuzkoa_Sports_2010/38878', 'https://www.365chess.com/tournaments/WCh_Women_Playoff_2010/38875', 'https://www.365chess.com/tournaments/IV_Stadium_Casablanca_Open_2010/38884', 'https://www.365chess.com/tournaments/Azerbaijanian_Solidarity_Day_2010/38876', 'https://www.365chess.com/tournaments/63rd_ch-RUS_Playoff_2010/38874', 'https://www.365chess.com/tournaments/Schaakfestival_Open_A_2010/38879', 'https://www.365chess.com/tournaments/Amitan_Memorial_2010/38880', 'https://www.365chess.com/tournaments/European_Rapid_Championship_Playoff_2010/38862', 'https://www.365chess.com/tournaments/European_Rapid_Championship_Playoff_3rd-4th_2010/38863', 'https://www.365chess.com/tournaments/29th_

Qui quello di chatgpt per accedere ai tornei

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def retrieve_tournament_tables(url):
    # Send a GET request to the URL and get the HTML response
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables in the HTML
    tables = soup.find_all('table')

    # Initialize an empty list to store dataframes
    dfs = []

    # Loop through each table and convert it to a Pandas dataframe
    for table in tables:
        # Check if the table contains rows with "th" elements (table headers)
        if table.find('tr', {'class': 'th'}):
            df = pd.read_html(str(table), header=0)[0]
        else:
            df = pd.read_html(str(table))
            if len(df) > 0:
                df = df[0]
            else:
                continue
        dfs.append(df)

    # Check if there are more pages in the tournament
    next_page_link = soup.find('a', {'class': 'pager-next'})
    if next_page_link:
        next_page_url = next_page_link.get('href')
        # Recursively call the function to retrieve tables from the next page
        dfs += retrieve_tournament_tables(next_page_url)

    if len(dfs) == 0:
        # Return an empty dataframe if no tables are found
        return pd.DataFrame()
    
    # Concatenate all dataframes into a single dataframe
    result_df = pd.concat(dfs)

    return dfs

# URL of the tournament
url = 'https://www.365chess.com/tournaments/29th_European_Blitz_2010/38864'

# Call the function to retrieve tables from all pages of the tournament
result_df = retrieve_tournament_tables(url)

# Print the final dataframe
result_df


In [ ]:
result_df

[           Date  Players  Elo Average  Games
 0  Dec 18, 2010       47         2597    169,
     Unnamed: 0  Name                      Name.1   ELO  Points
 0          NaN   NaN  GM Vachier Lagrave, Maxime  2703    17.0
 1          NaN   NaN       GM Ponomariov, Ruslan  2744    13.5
 2          NaN   NaN      GM Wojtaszek, Radoslaw  2726    11.5
 3          NaN   NaN        GM Rublevsky, Sergei  2678    10.0
 4          NaN   NaN        GM Inarkiev, Ernesto  2667     9.5
 5          NaN   NaN          GM Gashimov, Vugar  2733     9.0
 6          NaN   NaN        GM Ivanchuk, Vassily  2764     8.5
 7          NaN   NaN          GM Fridman, Daniel  2645     8.5
 8          NaN   NaN      GM Tkachiev, Vladislav  2636     8.5
 9          NaN   NaN           GM Jobava, Baadur  2707     7.0
 10         NaN   NaN        GM Movsesian, Sergei  2721     6.0
 11         NaN   NaN            GM Polgar, Judit  2686     6.0
 12         NaN   NaN           GM Korobov, Anton  2684     5.5
 13        

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def retrieve_tournament_tables(url):
    # Send a GET request to the URL and get the HTML response
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables in the HTML
    tables = soup.find_all('table')

    # Initialize a list to store dataframes for each table type
    dfs_by_table = [[] for _ in range(len(tables))]

    # Loop through each table and convert it to a Pandas dataframe
    for table_index, table in enumerate(tables):
        # Check if the table contains rows with "th" elements (table headers)
        if table.find('tr', {'class': 'th'}):
            df = pd.read_html(str(table), header=0)[0]
        else:
            df = pd.read_html(str(table))
            if len(df) > 0:
                df = df[0]
            else:
                continue

        # Append the dataframe to the list for its table type
        dfs_by_table[table_index].append(df)

    # Check if there are more pages in the tournament
    next_page_link = soup.find('a', {'class': 'pager-next'})
    if next_page_link:
        next_page_url = next_page_link.get('href')
        # Recursively call the function to retrieve tables from the next page
        next_page_dfs_by_table = retrieve_tournament_tables(next_page_url)

        # Concatenate the dataframes for each table type across pages, if they have the same number of columns
        for table_index, dfs in enumerate(dfs_by_table):
            if len(dfs) > 0 and len(next_page_dfs_by_table[table_index]) > 0 and dfs[0].shape[1] == next_page_dfs_by_table[table_index][0].shape[1]:
                dfs_by_table[table_index][0] = pd.concat([dfs[0], next_page_dfs_by_table[table_index][0]], ignore_index=True)

    # Concatenate all dataframes into a single dataframe
    result_df = pd.concat([df for dfs in dfs_by_table for df in dfs])

    return result_df

# URL of the tournament
url = 'https://www.365chess.com/tournaments/29th_European_Blitz_2010/38864'

# Call the function to retrieve tables from all pages of the tournament
result_df = retrieve_tournament_tables(url)

# Print the final dataframe
result_df


,Date,Players,Elo Average,Games,Unnamed: 0,Name,Name.1,ELO,Points,White,Black,Result,Round,Opening,View,0,1
0,"Dec 18, 2010",47.0,2597.0,169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,"GM Vachier Lagrave, Maxime",2703.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,"GM Ponomariov, Ruslan",2744.0,13.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,"GM Wojtaszek, Radoslaw",2726.0,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,"GM Rublevsky, Sergei",2678.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,"Dec 17, 2010",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Ivanchuk, V","Bologan, V",1-0,9.0,"A30 English, symmetrical variation",NaN,NaN,NaN
99,"Dec 17, 2010",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rublevsky, S","Gajewski, G",0-1,9.0,B40 Sicilian defence,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1, 2 Next",NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def retrieve_tournament_tables(url):
    # Send a GET request to the URL and get the HTML response
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables in the HTML
    tables = soup.find_all('table')

    # Initialize lists to store dataframes for each table type
    table_1_dfs = []
    table_2_dfs = []
    table_3_dfs = []

    # Loop through each table and convert it to a Pandas dataframe
    for table_index, table in enumerate(tables):
        # Check if the table contains rows with "th" elements (table headers)
        if table.find('tr', {'class': 'th'}):
            df = pd.read_html(str(table), header=0)#[0]
        else:
            df = pd.read_html(str(table))
            if len(df) > 0:
                df = df[0]
            else:
                continue

        # Append the dataframe to the appropriate list based on its table type
        if table_index == 0:
            table_1_dfs.append(df)
        elif table_index == 1:
            table_2_dfs.append(df)
        else:
            table_3_dfs.append(df)

    # Check if there are more pages in the tournament
    next_page_link = soup.find('a', {'class': 'pager-next'})
    if next_page_link:
        next_page_url = next_page_link.get('href')
        # Recursively call the function to retrieve tables from the next page
        next_page_table_3_dfs = retrieve_tournament_tables(next_page_url)[2]

        # Concatenate the dataframes for table 3 across pages, if they have the same number of columns
        if len(table_3_dfs) > 0 and len(next_page_table_3_dfs) > 0 and table_3_dfs[0].shape[1] == next_page_table_3_dfs[0].shape[1]:
            table_3_dfs[0] = pd.concat([table_3_dfs[0], next_page_table_3_dfs[0]], ignore_index=True)

    # Concatenate table 3 dataframes into a single dataframe
    table_3_df = pd.concat(table_3_dfs)

    return table_1_dfs, table_2_dfs, table_3_df

# URL of the tournament
url = 'https://www.365chess.com/tournaments/29th_European_Blitz_2010/38864'

# Call the function to retrieve tables from all pages of the tournament
table_1_dfs, table_2_dfs, table_3_df = retrieve_tournament_tables(url)

# Print the dataframes
print(table_1_dfs)
print(table_2_dfs)
print(table_3_df)


[           Date  Players  Elo Average  Games
0  Dec 18, 2010       47         2597    169]
[    Unnamed: 0  Name                      Name.1   ELO  Points
0          NaN   NaN  GM Vachier Lagrave, Maxime  2703    17.0
1          NaN   NaN       GM Ponomariov, Ruslan  2744    13.5
2          NaN   NaN      GM Wojtaszek, Radoslaw  2726    11.5
3          NaN   NaN        GM Rublevsky, Sergei  2678    10.0
4          NaN   NaN        GM Inarkiev, Ernesto  2667     9.5
5          NaN   NaN          GM Gashimov, Vugar  2733     9.0
6          NaN   NaN        GM Ivanchuk, Vassily  2764     8.5
7          NaN   NaN          GM Fridman, Daniel  2645     8.5
8          NaN   NaN      GM Tkachiev, Vladislav  2636     8.5
9          NaN   NaN           GM Jobava, Baadur  2707     7.0
10         NaN   NaN        GM Movsesian, Sergei  2721     6.0
11         NaN   NaN            GM Polgar, Judit  2686     6.0
12         NaN   NaN           GM Korobov, Anton  2684     5.5
13         NaN   NaN     

In [ ]:
table_3_df[21:99].drop(['Unnamed: 0', 'Name', 'Name.1', 'ELO', 'Points', 'View', 0, 1], axis=1)

,White,Black,Result,Round,Date,Opening
0,"Inarkiev, E","Ivanov, A",1-0,3.0,"Dec 17, 2010","B66 Sicilian, Richter-Rauzer, Rauzer attack, 7..."
1,"Gashimov, V","Stupak, K",1-0,3.0,"Dec 17, 2010",C00 French defence
2,"Bagrationi, A","Korobov, A",0-1,3.0,"Dec 17, 2010","E92 King's Indian, classical variation"
3,"Rublevsky, S","Okhotnik, V",1-0,3.0,"Dec 17, 2010","B29 Sicilian, Nimzovich-Rubinstein variation"
4,"Naiditsch, A","Bernasek, J",1-0,3.0,"Dec 17, 2010","B42 Sicilian, Kan, 5.Bd3"
...,...,...,...,...,...,...
73,"Ponomariov, R","Movsesian, S",1-0,7.0,"Dec 17, 2010","D37 Queen's Gambit Declined, 4.Nf3"
74,"Jobava, B","Tregubov, P",1-0,7.0,"Dec 17, 2010","B23 Sicilian, closed"
75,"Tkachiev, V","Inarkiev, E",1-0,7.0,"Dec 17, 2010",E60 King's Indian defence
76,"Wojtaszek, R","Gashimov, V",1-0,7.0,"Dec 17, 2010",A61 Benoni defence


In [ ]:
url_torneo_2010 = 'https://www.365chess.com/tournaments/29th_European_Blitz_2010/38864'
stringa_aggiuntiva = '/?p=1&start='

df = []
